# 🧪 Projet : Application de Traduction Multimodèle avec Évaluation

## 🎯 Objectif

- Créer une application de traduction multilingue utilisant plusieurs modèles de LLM.
- Comparer leurs performances avec des métriques standard.

---

## ✅ Tâches à réaliser

1. 🔍 **Rechercher un jeu de données de traduction de référence**  
   Le dataset doit contenir :
   - du **texte original**
   - une **traduction de référence**  
   👉 Sources possibles : Hugging Face, Kaggle, GitHub, etc.

2. 🧠 **Créer une application de traduction démo**  
   Utiliser :
   - **Deux modèles LLM de Ollama**
   - **Deux modèles LLM de Hugging Face**  
   👉 Langue au choix selon le dataset (ex : anglais → français)

3. 🎛️ **Utiliser différents niveaux de température** pour la génération :
   - `0`, `0.2`, `0.5`, `0.8`, `1.0`

4. 📊 **Évaluer les résultats** de traduction en utilisant :
   - **BLEU**
   - **ROUGE**

5. 💻 **Déployer une application interactive avec Gradio**  
   - L’application doit tourner en local

6. 🚀 **Publier le projet sur GitHub** :
   - Code complet + `requirements.txt`
   - Rapport de comparaison des performances des modèles
   - Vidéo démonstration de l’application
   - Lien vers le repo à partager

---

## 🧩 Technologies suggérées

- [🤗 Transformers – Hugging Face](https://huggingface.co/transformers/)
- [Ollama – LLMs locaux](https://ollama.com/)
- [Gradio – Interface utilisateur](https://www.gradio.app/)
- `nltk`, `rouge_score` pour l'évaluation
- Python, Jupyter, GitHub

---



# 0) Importations des bibliothèques + chemin

In [1]:
import subprocess
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, MBartForConditionalGeneration, MBart50Tokenizer
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
import gradio as gr
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

OLLAMA_PATH = os.getenv("OLLAMA_PATH")
if not OLLAMA_PATH:
    raise EnvironmentError("OLLAMA_PATH n'est pas défini dans le fichier .env")

In [8]:
import subprocess

OLLAMA_PATH = r"C:\Users\merwa\AppData\Local\Programs\Ollama\ollama.exe"

try:
    result = subprocess.run([OLLAMA_PATH, "--version"], capture_output=True, text=True)
    print(result.stdout)
except Exception as e:
    print(f"Erreur : {e}")

ollama version is 0.9.2



# 1) Chargement du dataset wmt14

In [3]:
#Charger dataset WMT14 fr-en
dataset = load_dataset("wmt14", "fr-en")

def get_en_fr_pairs(dataset, split='test', limit=10):
    subset = dataset[split].select(range(limit)) 
    sources = [item['translation']['en'] for item in subset]
    references = [item['translation']['fr'] for item in subset]
    return sources, references

sources, references = get_en_fr_pairs(dataset, limit=10)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

# 2) Choisir et utiliser 2 modèles Hugging Face + 2 modèles Ollama

| Modèle                               | Taille (paramètres) | Usage principal         | Notes                         |
| ------------------------------------ | ------------------- | ----------------------- | ----------------------------- |
| Helsinki-NLP/opus-mt-en-fr           | \~60M               | Traduction EN→FR légère | Rapide, petit, bon compromis  |
| facebook/mbart-large-50-many-to-many | \~610M              | Traduction multilingue  | Plus lourd, meilleure qualité |

| Critère           | Mistral 7B            | LLaMA 3.2           |
|-------------------|-----------------------|---------------------|
| Taille            | ~4.1 Go               | ~2 Go               |
| Paramètres        | ~7 milliards          | ~7 milliards        |
| RAM/VRAM requise  | ~7 Go (float16)       | ~4 Go (float16)     |
| Vitesse           | Modérée               | Plus rapide         |
| Disponibilité     | Oui via Ollama        | Oui via Ollama      |
| Usage principal   | Traduction, texte     | Chat, traduction    |
| Facilité d’usage  | Moyenne               | Moyenne             |

In [4]:
import torch
print("✅ PyTorch version :", torch.__version__)
print("📦 GPU dispo :", torch.cuda.is_available())

✅ PyTorch version : 2.7.1+cpu
📦 GPU dispo : False


In [5]:
#Charger modèles Hugging Face
model_1_name = "Helsinki-NLP/opus-mt-en-fr"
model_2_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer_1 = MarianTokenizer.from_pretrained(model_1_name)
model_1 = MarianMTModel.from_pretrained(model_1_name).to("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_2 = MBart50Tokenizer.from_pretrained(model_2_name)
model_2 = MBartForConditionalGeneration.from_pretrained(model_2_name).to("cuda" if torch.cuda.is_available() else "cpu")

def translate_hf(model, tokenizer, texts, temperature=0.0):
    device = next(model.parameters()).device
    if "mbart" in tokenizer.name_or_path.lower():
        tokenizer.src_lang = "en_XX"
        tokenizer.tgt_lang = "fr_XX"
        inputs = tokenizer(texts, return_tensors="pt", padding=True).to(device)
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"],
            do_sample=temperature > 0,
            temperature=temperature if temperature > 0 else None,
            num_beams=5 if temperature == 0 else 1,
            max_length=128,
        )
    else:
        inputs = tokenizer(texts, return_tensors="pt", padding=True).to(device)
        outputs = model.generate(
            **inputs,
            do_sample=temperature > 0,
            temperature=temperature if temperature > 0 else None,
            num_beams=5 if temperature == 0 else 1,
            max_length=128,
        )
    translated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return translated

def translate_ollama(model_name, text, temperature=0.0):
    """
    Traduction via Ollama avec variation du style selon la température,
    mais consigne claire : retourner uniquement la traduction, sans aucun commentaire.
    """

    #Valeur par défaut de secours (sécurité)
    base_prompt = "Translate the following English text to French."

    if temperature == 0.0:
        base_prompt = "Translate the following English text to French. Be precise and literal."
    elif temperature <= 0.2:
        base_prompt = "Translate the following English text to French with natural phrasing."
    elif temperature <= 0.5:
        base_prompt = "Translate the following English text to French with a balance between literal and creative style."
    elif temperature <= 0.8:
        base_prompt = "Translate the following English text to French in a more creative and expressive way."
    elif temperature <= 1.0:
        base_prompt = "Translate the following English text to French in a free, creative way, using expressive French."

    #Ajout de la consigne stricte selon le modèle
    if model_name.startswith("mistral"):
        base_prompt += "\nDo not include any explanation, greeting, or formatting. Just return the French translation as plain text only. Nothing else."
    else:
        base_prompt += "\nReturn ONLY the French translation. No explanations or formatting."

    #Construction finale du prompt
    prompt = f"{base_prompt}\n\n{text}"

    command = [
        OLLAMA_PATH,
        "run",
        model_name,
    ]
    try:
        result = subprocess.run(command, input=prompt, capture_output=True, text=True, encoding="utf-8", timeout=20)
        if result.returncode != 0:
            return f"Error: {result.stderr.strip()}"
        return result.stdout.strip()
    except Exception as e:
        return f"Exception: {str(e)}"

c:\Users\merwa\Documents\GitHub\merwan.boudrias\projet_trad\venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


# 3) Utilisation de différentes températures

## Explication des métriques

### BLEU (Bilingual Evaluation Understudy)

**Qu’est-ce que c’est ?**  
BLEU mesure la similarité entre la traduction automatique et une ou plusieurs traductions de référence humaines.

**Comment ça marche ?**  
Il compte la correspondance des n-grammes (groupes de mots) dans la traduction générée par rapport à la référence.

**Interprétation :**

- 0-10 : Qualité très faible, traduction presque incompréhensible ou hors sujet.  
- 10-30 : Traduction compréhensible mais avec plusieurs erreurs.  
- 30-50 : Bonne traduction, souvent utilisable, quelques erreurs mineures.  
- >50 : Traduction de très bonne qualité, proche d’une traduction humaine.

**Attention :**  
BLEU ne capture pas toujours la fluidité ou la qualité sémantique parfaite.

---

### ROUGE-L (Recall-Oriented Understudy for Gisting Evaluation - Longest common subsequence)

**Qu’est-ce que c’est ?**  
ROUGE-L mesure la qualité de la traduction en évaluant la plus longue sous-séquence commune entre la traduction générée et la référence.

**Comment ça marche ?**  
Il évalue la structure globale, la cohérence et le contenu partagé, prenant en compte la fluidité plus que BLEU.

**Valeurs typiques :**

- 0.0 à 1.0 (ou 0% à 100%), où 1 signifie correspondance parfaite.

**Interprétation :**

- <0.3 : Faible correspondance, traduction de mauvaise qualité.  
- 0.3 - 0.6 : Moyenne à bonne qualité.  
- >0.6 : Très bonne qualité, bonne fluidité et correspondance.


In [9]:
#Évaluation automatique BLEU & ROUGE
def evaluate_translations(references, predictions):
    #BLEU
    bleu = corpus_bleu(predictions, [references])
    
    #ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1':0, 'rouge2':0, 'rougeL':0}
    for ref, pred in zip(references, predictions):
        scores = scorer.score(ref, pred)
        for key in rouge_scores.keys():
            rouge_scores[key] += scores[key].fmeasure
    #Moyenne
    n = len(references)
    for key in rouge_scores:
        rouge_scores[key] /= n
    return bleu.score, rouge_scores

#Tester différents modèles avec températures
temperatures = [0.0, 0.2, 0.5, 0.8, 1.0]

results = {
    "huggingface_helsinki": {},
    "huggingface_mbart": {},
    "ollama_llama3.2": {},
    "ollama_mistral7b": {},
}

print("Traductions sur quelques phrases phrases du test WMT14...\n")

#Hugging Face
for temp in temperatures:
    print(f"HF Helsinki - temp={temp}")
    pred1 = translate_hf(model_1, tokenizer_1, sources, temperature=temp)
    results["huggingface_helsinki"][temp] = pred1
    bleu_score, rouge_scores = evaluate_translations(references, pred1)
    print(f"BLEU={bleu_score:.2f}, ROUGE-L={rouge_scores['rougeL']:.3f}")

    print(f"HF mBART - temp={temp}")
    pred2 = translate_hf(model_2, tokenizer_2, sources, temperature=temp)
    results["huggingface_mbart"][temp] = pred2
    bleu_score, rouge_scores = evaluate_translations(references, pred2)
    print(f"BLEU={bleu_score:.2f}, ROUGE-L={rouge_scores['rougeL']:.3f}\n")

#Ollama avec températures simulées dans le prompt
for temp in temperatures:
    print(f"Ollama llama3.2 - temp={temp}")
    preds_llama = [translate_ollama("llama3.2:latest", text, temperature=temp) for text in sources]
    results["ollama_llama3.2"][temp] = preds_llama
    bleu_score, rouge_scores = evaluate_translations(references, preds_llama)
    print(f"BLEU={bleu_score:.2f}, ROUGE-L={rouge_scores['rougeL']:.3f}\n")

for temp in temperatures:
    print(f"Ollama mistral:7b - temp={temp}")
    preds_mistral = [translate_ollama("mistral:7b", text, temperature=temp) for text in sources]
    results["ollama_mistral7b"][temp] = preds_mistral
    bleu_score, rouge_scores = evaluate_translations(references, preds_mistral)
    print(f"BLEU={bleu_score:.2f}, ROUGE-L={rouge_scores['rougeL']:.3f}\n")

Traductions sur quelques phrases phrases du test WMT14...

HF Helsinki - temp=0.0
BLEU=31.01, ROUGE-L=0.586
HF mBART - temp=0.0
BLEU=26.65, ROUGE-L=0.549

HF Helsinki - temp=0.2


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


BLEU=27.54, ROUGE-L=0.563
HF mBART - temp=0.2
BLEU=29.38, ROUGE-L=0.563

HF Helsinki - temp=0.5


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


BLEU=29.89, ROUGE-L=0.536
HF mBART - temp=0.5
BLEU=20.78, ROUGE-L=0.504

HF Helsinki - temp=0.8


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


BLEU=14.93, ROUGE-L=0.437
HF mBART - temp=0.8
BLEU=17.13, ROUGE-L=0.452

HF Helsinki - temp=1.0


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


BLEU=19.43, ROUGE-L=0.441
HF mBART - temp=1.0
BLEU=10.67, ROUGE-L=0.390

Ollama llama3.2 - temp=0.0
BLEU=24.14, ROUGE-L=0.519

Ollama llama3.2 - temp=0.2
BLEU=20.26, ROUGE-L=0.490

Ollama llama3.2 - temp=0.5
BLEU=28.88, ROUGE-L=0.547

Ollama llama3.2 - temp=0.8
BLEU=21.39, ROUGE-L=0.436

Ollama llama3.2 - temp=1.0
BLEU=16.04, ROUGE-L=0.445

Ollama mistral:7b - temp=0.0
BLEU=21.43, ROUGE-L=0.458

Ollama mistral:7b - temp=0.2
BLEU=23.08, ROUGE-L=0.454

Ollama mistral:7b - temp=0.5
BLEU=25.70, ROUGE-L=0.512

Ollama mistral:7b - temp=0.8
BLEU=20.73, ROUGE-L=0.428

Ollama mistral:7b - temp=1.0
BLEU=20.69, ROUGE-L=0.488



# 4) Métriques d'évaluations (BLEU + ROUGE-L)

## Résultats de l'évaluation sur 10 phrases du jeu de test WMT14 (EN → FR)

| Modèle                | Température | BLEU  | ROUGE-L |
| --------------------- | ----------- | ----- | ------- |
| **HF Helsinki**       | 0.0         | 31.01 | 0.586   |
|                       | 0.2         | 27.54 | 0.563   |
|                       | 0.5         | 29.89 | 0.536   |
|                       | 0.8         | 14.93 | 0.437   |
|                       | 1.0         | 19.43 | 0.441   |
| **HF mBART**          | 0.0         | 26.65 | 0.549   |
|                       | 0.2         | 29.38 | 0.563   |
|                       | 0.5         | 20.78 | 0.504   |
|                       | 0.8         | 17.13 | 0.452   |
|                       | 1.0         | 10.67 | 0.390   |
| **Ollama llama3.2**   | 0.0         | 24.14 | 0.519   |
|                       | 0.2         | 20.26 | 0.490   |
|                       | 0.5         | 28.88 | 0.547   |
|                       | 0.8         | 21.39 | 0.436   |
|                       | 1.0         | 16.04 | 0.445   |
| **Ollama mistral:7b** | 0.0         | 21.43 | 0.458   |
|                       | 0.2         | 23.08 | 0.454   |
|                       | 0.5         | 25.70 | 0.512   |
|                       | 0.8         | 20.73 | 0.428   |
|                       | 1.0         | 20.69 | 0.488   |


---

## Explication des métriques

### 🔵 BLEU (Bilingual Evaluation Understudy)

**Définition :**  
BLEU mesure la similarité entre la traduction automatique et une ou plusieurs traductions humaines de référence.

**Comment ça fonctionne ?**  
Il compare des groupes de mots (n-grammes) entre la sortie du modèle et la référence, en pénalisant les traductions trop courtes.

**Interprétation des scores :**

| Score BLEU | Qualité de traduction                   |
|------------|------------------------------------------|
| 0–10       | Très faible (souvent incompréhensible)  |
| 10–30      | Moyen (compréhensible avec erreurs)      |
| 30–50      | Bon (utilisable, erreurs mineures)       |
| >50        | Excellent (proche d’une traduction humaine) |

---

### 🔴 ROUGE-L (Longest Common Subsequence)

**Définition :**  
ROUGE-L mesure la qualité de la traduction en évaluant la **plus longue sous-séquence commune** entre la prédiction et la référence.

**Avantage :**  
Il capte la **structure globale et la fluidité** du texte (meilleur complément au BLEU).

**Interprétation des scores :**

| Score ROUGE-L | Qualité de traduction                    |
|---------------|-------------------------------------------|
| <0.3          | Faible qualité, structure incohérente     |
| 0.3 – 0.6     | Moyenne à bonne qualité                   |
| >0.6          | Très bonne qualité et bonne cohérence     |

---

👉 **Remarque :**  
Les scores peuvent varier selon la longueur des textes, la complexité sémantiqu


# 5) Gradio 

In [10]:
#Démo Gradio corrigée
def demo_translate(text, model_choice, temperature):
    if not text or text.strip() == "":
        return "Veuillez saisir un texte !"  
    if model_choice == "Helsinki-NLP/opus-mt-en-fr":
        translation = translate_hf(model_1, tokenizer_1, [text], temperature=temperature)[0]
    elif model_choice == "facebook/mbart-large-50-many-to-many-mmt":
        translation = translate_hf(model_2, tokenizer_2, [text], temperature=temperature)[0]
    elif model_choice == "llama3.2":
        translation = translate_ollama("llama3.2:latest", text, temperature=temperature)
    elif model_choice == "mistral:7b":
        translation = translate_ollama("mistral:7b", text, temperature=temperature)
    else:
        translation = "Modèle inconnu"
    return translation

iface = gr.Interface(
    fn=demo_translate,
    inputs=[
        gr.Textbox(lines=3, label="Texte anglais à traduire"),
        gr.Dropdown(
            choices=["Helsinki-NLP/opus-mt-en-fr", "facebook/mbart-large-50-many-to-many-mmt", "llama3.2", "mistral:7b"],
            label="Modèle"
        ),
        gr.Dropdown(
            choices=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
            label="Température",
            value=0
        )
    ],
    outputs="text",
    title="Démo traduction anglais→français",
    description="Testez la traduction en temps réel avec Hugging Face et Ollama (température simulée dans Ollama via prompt)",
    
)

if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
